## Business Impact Analysis

In this notebook, we shift the focus from technical modeling to business implications. We will be analysing how key metrics such as revenue, inventory turnover, and price elasticity affect Walmart’s operations. Using sales data, I estimate product- and store-level price elasticity to understand customer sensitivity to price changes. I also examine how SNAP participation impacts demand. These insights help inform data-driven decisions for pricing, promotions, inventory planning, and store-level strategy.

In [2]:
import warnings
import gc
import numpy as np
import pandas as pd
from scipy.stats import linregress
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder
from pandas.errors import SettingWithCopyWarning

In [4]:
sales = pd.read_csv(r"C:\Users\user\Desktop\sales_train_validation.csv")
calendar = pd.read_csv(r"C:\Users\user\Desktop\calendar.csv")
prices = pd.read_csv(r"C:\Users\user\Desktop\sell_prices.csv")
submission = pd.read_csv(r"C:\Users\user\Desktop\sample_submission.csv")

In [5]:
def reduce_mem_usage(df, verbose=False):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    int_columns = df.select_dtypes(include=["int"]).columns
    float_columns = df.select_dtypes(include=["float"]).columns

    for col in int_columns:
        df[col] = pd.to_numeric(df[col], downcast="integer")

    for col in float_columns:
        df[col] = pd.to_numeric(df[col], downcast="float")

    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df
    
reduce_mem_usage(sales)
reduce_mem_usage(calendar)
reduce_mem_usage(prices)
reduce_mem_usage(submission)

print("sales shape:", sales.shape)
print("prices shape:", prices.shape)
print("calendar shape:", calendar.shape)
print("submission shape:", submission.shape)

sales shape: (30490, 1919)
prices shape: (6841121, 4)
calendar shape: (1969, 14)
submission shape: (60980, 29)


In [7]:
NUM_ITEMS = sales.shape[0]  
DAYS_PRED = submission.shape[1] - 1

def encode_categorical(df, cols):
    for col in cols:
        le = LabelEncoder()
        not_null = df[col][df[col].notnull()]
        df[col] = pd.Series(le.fit_transform(not_null), index=not_null.index)

    return df

calendar = encode_categorical(
    calendar, ["event_name_1", "event_type_1", "event_name_2", "event_type_2"]
).pipe(reduce_mem_usage)

sales = encode_categorical(
    sales, ["item_id", "dept_id", "cat_id", "store_id", "state_id"],
).pipe(reduce_mem_usage)

prices = encode_categorical(prices, ["item_id", "store_id"]).pipe(reduce_mem_usage)

In [9]:
def extract_num(ser):
    return ser.str.extract(r"(\d+)").astype(np.int16)

def reshape_sales(sales, submission, d_thresh=0, verbose=True):
    id_columns = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]

    product = sales[id_columns]

    sales = sales.melt(id_vars=id_columns, var_name="d", value_name="demand",)
    sales = reduce_mem_usage(sales)

    vals = submission[submission["id"].str.endswith("validation")]
    evals = submission[submission["id"].str.endswith("evaluation")]

    vals.columns = ["id"] + [f"d_{d}" for d in range(1914, 1914 + DAYS_PRED)]
    evals.columns = ["id"] + [f"d_{d}" for d in range(1942, 1942 + DAYS_PRED)]

    evals["id"] = evals["id"].str.replace("_evaluation", "_validation")
    vals = vals.merge(product, how="left", on="id")
    evals = evals.merge(product, how="left", on="id")
    evals["id"] = evals["id"].str.replace("_validation", "_evaluation")

    if verbose:
        print("validation")
        display(vals)

        print("evaluation")
        display(evals)

    vals = vals.melt(id_vars=id_columns, var_name="d", value_name="demand")
    evals = evals.melt(id_vars=id_columns, var_name="d", value_name="demand")

    sales["part"] = "train"
    vals["part"] = "validation"
    evals["part"] = "evaluation"

    data = pd.concat([sales, vals, evals], axis=0)

    del sales, vals, evals

    data["d"] = extract_num(data["d"])
    data = data[data["d"] >= d_thresh]

    data = data[data["part"] != "evaluation"]

    gc.collect()

    if verbose:
        print("data")
        display(data)

    return data


def merge_calendar(data, calendar):
    calendar = calendar.drop(["weekday", "wday", "month", "year"], axis=1)
    return data.merge(calendar, how="left", on="d")


def merge_prices(data, prices):
    return data.merge(prices, how="left", on=["store_id", "item_id", "wm_yr_wk"])

In [11]:
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

data = reshape_sales(sales, submission, d_thresh=1941 - int(365 * 2))
del sales
gc.collect()

calendar["d"] = extract_num(calendar["d"])
data = merge_calendar(data, calendar)
del calendar
gc.collect()

data = merge_prices(data, prices)
del prices
gc.collect()

data = reduce_mem_usage(data)

validation


,id,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,...,d_1937,d_1938,d_1939,d_1940,d_1941,item_id,dept_id,cat_id,store_id,state_id
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1437,3,1,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1438,3,1,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1439,3,1,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1440,3,1,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1441,3,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1432,2,0,9,2
30486,FOODS_3_824_WI_3_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1433,2,0,9,2
30487,FOODS_3_825_WI_3_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1434,2,0,9,2
30488,FOODS_3_826_WI_3_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1435,2,0,9,2


evaluation


,id,d_1942,d_1943,d_1944,d_1945,d_1946,d_1947,d_1948,d_1949,d_1950,...,d_1965,d_1966,d_1967,d_1968,d_1969,item_id,dept_id,cat_id,store_id,state_id
0,HOBBIES_1_001_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1437,3,1,0,0
1,HOBBIES_1_002_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1438,3,1,0,0
2,HOBBIES_1_003_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1439,3,1,0,0
3,HOBBIES_1_004_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1440,3,1,0,0
4,HOBBIES_1_005_CA_1_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1441,3,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1432,2,0,9,2
30486,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1433,2,0,9,2
30487,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1434,2,0,9,2
30488,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1435,2,0,9,2


data


,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,part
36892900,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,1211,1,train
36892901,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,1211,0,train
36892902,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0,1211,0,train
36892903,HOBBIES_1_004_CA_1_validation,1440,3,1,0,0,1211,1,train
36892904,HOBBIES_1_005_CA_1_validation,1441,3,1,0,0,1211,2,train
...,...,...,...,...,...,...,...,...,...
853715,FOODS_3_823_WI_3_validation,1432,2,0,9,2,1941,0,validation
853716,FOODS_3_824_WI_3_validation,1433,2,0,9,2,1941,0,validation
853717,FOODS_3_825_WI_3_validation,1434,2,0,9,2,1941,0,validation
853718,FOODS_3_826_WI_3_validation,1435,2,0,9,2,1941,0,validation


In [13]:
def add_demand_features(df):
    for diff in [0, 1, 2]:
        shift = DAYS_PRED + diff
        df[f"shift_t{shift}"] = df.groupby(["id"])["demand"].transform(
            lambda x: x.shift(shift)
        )

    for window in [7, 30, 60, 90, 180]:
        df[f"rolling_std_t{window}"] = df.groupby(["id"])["demand"].transform(
            lambda x: x.shift(DAYS_PRED).rolling(window).std()
        )

    for window in [7, 30, 60, 90, 180]:
        df[f"rolling_mean_t{window}"] = df.groupby(["id"])["demand"].transform(
            lambda x: x.shift(DAYS_PRED).rolling(window).mean()
        )

    for window in [7, 30, 60]:
        df[f"rolling_min_t{window}"] = df.groupby(["id"])["demand"].transform(
            lambda x: x.shift(DAYS_PRED).rolling(window).min()
        )

    for window in [7, 30, 60]:
        df[f"rolling_max_t{window}"] = df.groupby(["id"])["demand"].transform(
            lambda x: x.shift(DAYS_PRED).rolling(window).max()
        )

    df["rolling_skew_t30"] = df.groupby(["id"])["demand"].transform(
        lambda x: x.shift(DAYS_PRED).rolling(30).skew()
    )
    df["rolling_kurt_t30"] = df.groupby(["id"])["demand"].transform(
        lambda x: x.shift(DAYS_PRED).rolling(30).kurt()
    )
    return df


def add_price_features(df):
    df["shift_price_t1"] = df.groupby(["id"])["sell_price"].transform(
        lambda x: x.shift(1)
    )
    df["price_change_t1"] = (df["shift_price_t1"] - df["sell_price"]) / (
        df["shift_price_t1"]
    )
    df["rolling_price_max_t365"] = df.groupby(["id"])["sell_price"].transform(
        lambda x: x.shift(1).rolling(365).max()
    )
    df["price_change_t365"] = (df["rolling_price_max_t365"] - df["sell_price"]) / (
        df["rolling_price_max_t365"]
    )

    df["rolling_price_std_t7"] = df.groupby(["id"])["sell_price"].transform(
        lambda x: x.rolling(7).std()
    )
    df["rolling_price_std_t30"] = df.groupby(["id"])["sell_price"].transform(
        lambda x: x.rolling(30).std()
    )
    return df.drop(["rolling_price_max_t365", "shift_price_t1"], axis=1)


def add_time_features(df, dt_col):
    df[dt_col] = pd.to_datetime(df[dt_col])
    attrs = [
        "year",
        "quarter",
        "month",
        "day",
        "dayofweek",
    ]

    for attr in attrs:
        dtype = np.int16 if attr == "year" else np.int8
        df[attr] = getattr(df[dt_col].dt, attr).astype(dtype)

    df["week"] = df[dt_col].dt.isocalendar().week.astype(np.int8)
    
    df["is_weekend"] = df["dayofweek"].isin([5, 6]).astype(np.int8)
    return df

In [15]:
data = add_demand_features(data).pipe(reduce_mem_usage)
data = add_price_features(data).pipe(reduce_mem_usage)
dt_col = "date"
data = add_time_features(data, dt_col).pipe(reduce_mem_usage)
data = data.sort_values("date")

print("start date:", data[dt_col].min())
print("end date:", data[dt_col].max())
print("data shape:", data.shape)

start date: 2014-05-23 00:00:00
end date: 2016-05-22 00:00:00
data shape: (22288190, 51)


Now let's talk about this project from a business's point of view. Walmart is a large retailer company that operates with thin margins and high sales volume. The primary businesss metrics relevant to this project could be the following:

- **Revenue**: Walmart tracks overall revenue at multiple levels: per store, per state, per product category, and overall. This metric is really a basic yet very important for retailer companies.

- **Gross Margin**: Gross margin is a key business metric, as Walmart relies on predicting demand to price items correctly and manage inventory costs.

- **Inventory Turnover**: This one also must be tracked in my opinion by Walmart's business team. It measures how efficiently inventory is used. High inventory turnover maximizes revenue while avoiding storage costs and obsolescence.

- **Stock-out Rate (Lost Sales)**: Percentage of items unavailable (out of stock) when customers try to purchase them. Forecasting demand prevents stock-outs.

- **Overstocking Costs**: Holding extra inventory results in higher warehousing costs, spoilage, and lost capital, so it's essential to predict demand accurately.

- **Sales per Square Foot (Store Efficiency)**: I think this one is one of the most important ones too, especially relevant to compare performance across stores of varying sizes.

- **Customer Satisfaction**: This one is a little bit general and harder to track metric, but really important to keep loyal customers shopping at Walmart.

- **SNAP-Specific Metrics**: For states where SNAP applies, Walmart may monitor how SNAP affects sales trends and revenue, as SNAP may cause temporary spikes in specific product categories.

Besides this general useful metrics, that I think Walmart could use track to improve their performance on market, let's also talk more our case scenario and say how our forecast improves these business metrics:

- **Revenue & Customer Satisfaction**: Accurate demand forecasting ensures that goods are available when customers shop. Meeting demand helps maintain customer satisfaction, loyalty, and overall revenue.

- **Inventory Turnover**: Overstocking reduces inventory turnover, gathering up financial resources. Forecasting avoids overstock through precise demand planning.

- **Stock-Out Rate & Lost Sales**: Avoiding understock situations ensures sales aren’t lost to competitors. Accurate predictions minimize stock-out rates.

- **Gross Margin Optimization**: Forecasting allows adjustments in pricing strategies, markdown management, and promotions to maximize gross margins.

- **Overstock Costs**: Accurate forecasting avoids excess inventory, which reduces carrying costs and avoids wastage for perishables.

- **SNAP Demand Trends**: Forecasting adjusts supply plans for products heavily bought by SNAP users. Walmart can adjust its stocking policies to match periodic demand spikes.

The metrics that we've just overviewed was more of a financial side, for business teams, now let's talk about more technical - data science, side:

So in competitions, machine learning models are often evaluated using general-purpose metrics like Root Mean Squared Error (RMSE). While RMSE is useful from a technical standpoint — measuring how close the predicted values are to the actual ones — it doesn’t always align with business objectives. RMSE simply penalizes large errors more than small ones, but it treats all products and errors equally. This is a limitation when applying it to a business like Walmart. For instance, incorrectly predicting sales of a high-margin item like a television has a much bigger financial impact than an error on low-cost items like bread. Yet, RMSE considers them the same. This can lead to model optimizations that don’t reflect Walmart’s actual priorities — such as revenue, profit, or customer satisfaction.

Let's say better metrics that are more aligned for this business problem:

1. Weighted RMSE

Instead of treating all items equally, this metric gives more importance to products that contribute more to revenue. That way, the model is encouraged to be more accurate on high-value items. This is more aligned with business interests, as it prioritizes accuracy where financial impact is greatest.

2. Stock-Out Penalizing Loss

This metric recognizes that under-predicting demand (leading to stock-outs) is often worse than over-predicting. Stock-outs can mean lost sales, unhappy customers, and damaged brand reputation. By penalizing under-forecasting more heavily, the model learns to avoid these situations, which helps maintain customer satisfaction and revenue.

3. Inventory Cost-Aware Error

This metric tries to balance the cost of overstocking and understocking. Overstocking leads to higher storage and potential waste, while understocking results in missed sales. By embedding these real costs into the metric, the model can make smarter trade-offs that reflect actual operational challenges, improving overall efficiency and profitability.

4. Mean Weighted Absolute Percentage Error (MWAPE)

This is a more flexible alternative that focuses on percentage errors while giving more weight to high-selling or high-revenue products. It's useful for businesses because it highlights relative accuracy while still emphasizing the products that matter most financially.

This metrics are little bit trickier to implement for our case scenario, but we are talking more generally, so if implemented, this few metrics could tell lot more to a data science team, and merit the company in a more convinient and simpler ways.

Feature Importance plays a big role for a decision-making process. We have LightGBM's feature importance in our hands and let's analyze it and speak from the business perspective:

**Rolling Averages and Standard Deviations**

- **Short and Medium Term Trends**  
  The 30-day and 7-day rolling means (rolling_mean_t30, rolling_mean_t7) were the most influential features. These capture:
  - Daily and weekly sales momentum.
  - Emerging trends at the SKU level.

- **Sales Volatility Awareness**  
  The 180-day rolling standard deviation (rolling_std_t180) shows historical fluctuations in demand. This allows Walmart to:
  - Conservatively stock high-volatility items to reduce risk of stock-outs or overstock.
  - Aggressively stock low-variance items with leaner inventory levels.

- **Strategic Action**  
  Tailor inventory strategy:
  - High-variance - buffer stock or demand smoothing.
  - Low-variance - lean inventory, faster turnover.


**Seasonality and Time-Based Features**

- Time Features like month, day, and week were important, revealing seasonal buying behavior.

- Holiday Sensitivity 
  Certain products (party supplies, decorations) experience spikes during:
  - Christmas, Black Friday, Mother’s Day, etc.

- Actionable Moves  
  Walmart can:
  - Schedule targeted marketing campaigns.
  - Allocate temporary labor to regions with seasonal surges.


**Price and Elasticity Features**

- Key Features: sell_price, price_change_t1, and similar metrics.

- Revenue Optimization 
  Walmart can:
  - Analyze price elasticity to find which products are sensitive to price.
  - Implement dynamic pricing strategies for revenue maximization.


**Rolling Trends Across Longer Timeframes**

- Long-Term Features: rolling_mean_t180, rolling_std_t60.

- Business Use Cases:
  - Spot products with sustained popularity for stocking priority.
  - Flag underperforming categories that may benefit from promotions or replacements.


**Event-Based and Regional Features**

- Localized Demand 
  Features like snap_WI, event_type_1, and state_id reflect:
  - Region-specific SNAP participation (Wisconsin).
  - Demand changes driven by events or holidays.

- **Implications**:
  - Stock more SNAP-eligible items in high-SNAP areas.
  - Align inventory and marketing around local events (sports games, Mother's Day).

So we talked about all the objectives, technical and non-technical metrics that was used and could have been used in other cases, now let's talk about how those metrics could help business teams for decision-making:

 **1. Variables Walmart Can Control**

- **Prices**  
  Walmart can modify product prices based on demand forecasts to optimize revenue. For example, they might:
  - Reduce prices during periods of low demand.
  - Adjust prices near stock exhaustion to increase turnover.

- **Promotions and Discounts**  
  Offering strategic discounts on overstocked items (especially regionally) can help clear inventory when demand is forecasted to be low.

- **Inventory Decisions**  
  Forecasts allow Walmart to:
  - Fine-tune ordering quantities.
  - Avoid overstock (waste/storage cost) and understock (lost sales).

- **Advertising Spend**  
  Advertising can be increased or optimized:
  - In locations where projected demand is low.
  - For products/categories that need a visibility boost.


 **2. Variables Walmart Cannot Control**

- **External Factors**  
  Elements outside Walmart’s control that influence demand include:
  - Weather conditions (water sales spike before hurricanes).
  - National holidays or events.
  - Macroeconomic trends (inflation, recession).

- **Customer Preferences**  
  Shifts in consumer taste and loyalty, such as:
  - Preference for competitor brands.
  - Changes in dietary, cultural, or ethical buying habits.


 **3. Strategic Decisions Walmart Can Make with the Model**

- **Optimize Replenishment**  
  Use accurate forecasts to:
  - Adjust store-level inventory.
  - Avoid delays and stock-outs.

- **Adjust Pricing Based on Elasticity**  
  Implement dynamic pricing strategies, such as:
  - Increasing prices for inelastic goods during high demand.
  - Offering promotions on elastic goods during slow periods.

- **Category-Wise Planning**  
  Allocate resources more effectively by:
  - Identifying underperforming categories for reevaluation.
  - Expanding shelf space for consistently high-performing items.

In [21]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,part,date,...,price_change_t365,rolling_price_std_t7,rolling_price_std_t30,year,quarter,month,day,dayofweek,week,is_weekend
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,1211,1,train,2014-05-23,...,NaN,NaN,NaN,2014,2,5,23,4,21,0
20334,FOODS_2_214_TX_3_validation,428,1,0,6,1,1211,0,train,2014-05-23,...,NaN,NaN,NaN,2014,2,5,23,4,21,0
20333,FOODS_2_213_TX_3_validation,427,1,0,6,1,1211,0,train,2014-05-23,...,NaN,NaN,NaN,2014,2,5,23,4,21,0
20332,FOODS_2_212_TX_3_validation,426,1,0,6,1,1211,0,train,2014-05-23,...,NaN,NaN,NaN,2014,2,5,23,4,21,0
20331,FOODS_2_211_TX_3_validation,425,1,0,6,1,1211,0,train,2014-05-23,...,NaN,NaN,NaN,2014,2,5,23,4,21,0


**Price Elasticity of Demand (PED)** is a way to measure how much the demand for a product changes when its price changes.

If a small change in price causes a big change in how much people buy, we say the product is price elastic. For example, if the price of a soft drink goes up, people might quickly buy less or switch to another brand.

If the demand stays nearly the same even after the price changes, the product is price inelastic. 

Let’s say Walmart wants to analyze the demand for one of its products:

**Step 1: Measure Sensitivity**

They observe sales data over time and notice that when the price of this detergent increases by $1, the number of units sold drops significantly. This tells them the product is price elastic—customers are sensitive to price changes and might switch to another brand or store.

**Step 2: Business Decisions Based on PED**

Pricing Strategy: Since the detergent is price elastic, Walmart might avoid raising the price too much. Instead, they could lower it slightly to attract more buyers and increase total sales.

Promotions: Walmart could run regular discounts or bundle offers, knowing that even small price cuts could boost demand and drive traffic to the store.

Supplier Negotiations: If Walmart knows that customers are price-sensitive for this product, they might push the supplier for lower wholesale prices, allowing Walmart to stay competitive without cutting into profits.

Inventory Planning: During promotional periods, Walmart can stock up, anticipating a sharp rise in demand due to the product's elasticity.

In [23]:
df = reduce_mem_usage(data)
product_id = "HOBBIES_1_001_CA_1_validation"
product_df = df[df["id"] == product_id]

product_df = product_df[(product_df["demand"] > 0) & (product_df["sell_price"] > 0)]

product_df["log_price"] = np.log(product_df["sell_price"])
product_df["log_demand"] = np.log(product_df["demand"])

slope, intercept, r_value, p_value, std_err = linregress(
    product_df["log_price"], product_df["log_demand"]
)

price_elasticity = slope 

print(f"Estimated price elasticity for {product_id}: {price_elasticity:.2f}")

Estimated price elasticity for HOBBIES_1_001_CA_1_validation: -2.02


In [24]:
store_elasticities = []

for store_id, store_df in data.groupby("store_id"):
    store_df = store_df[(store_df["demand"] > 0) & (store_df["sell_price"] > 0)]
    
    if store_df["sell_price"].nunique() > 1 and len(store_df) > 30:
        store_df["log_price"] = np.log(store_df["sell_price"])
        store_df["log_demand"] = np.log(store_df["demand"])

        try:
            slope, *_ = linregress(store_df["log_price"], store_df["log_demand"])
            store_elasticities.append({"store_id": store_id, "elasticity": slope})
        except ValueError:
            continue

store_elasticity_df = pd.DataFrame(store_elasticities)
store_elasticity_df

,store_id,elasticity
0,0,-0.363365
1,1,-0.295157
2,2,-0.378001
3,3,-0.301862
4,4,-0.323756
5,5,-0.322010
6,6,-0.313335
7,7,-0.352450
8,8,-0.402976
9,9,-0.375405


Let's discuss this results with business implications. First, we calculated the price elasticity of demand for a single product—HOBBIES_1_001_CA_1, and found it to be approximately -2.02. This indicates a highly elastic demand: a 1% increase in price leads to roughly a 2% drop in quantity demanded. Customers for this product are very sensitive to price changes, likely switching brands or postponing purchases if prices rise. Walmart should consider this when pricing such products to avoid losing significant sales volume.

Now let's check the store aggregated elastisities and discuss the implications of the first 3 stores as an example: 

- Store 0 (-0.36): Demand is relatively inelastic. Price changes have a moderate effect on sales volume. Walmart can afford small price increases here without losing many customers, helping increase margin.

- Store 1 (-0.30): Even less sensitive. Customers here are less price-conscious, perhaps due to limited local competition or brand loyalty. Walmart might focus less on aggressive price promotions and more on maintaining consistent pricing or bundling to maximize profit.

- Store 2 (-0.38): Slightly more sensitive but still inelastic. Walmart can use moderate discounts strategically to boost volume during key periods (holidays, weekends) without hurting profits too much.


How Walmart can use these insights:

- Tailored Pricing Strategies: Instead of applying uniform price changes across all stores, Walmart can customize pricing by store elasticity. For example, stores with more elastic demand might benefit from more frequent discounts or competitive pricing, while inelastic stores might sustain higher prices.

- Promotion Optimization: Stores with higher elasticity (closer to -1 or beyond) can run promotions to stimulate sales more effectively. For stores with low elasticity, promotions might focus on upselling or cross-selling rather than price cuts.

- Inventory Planning: Understanding elasticity helps Walmart forecast how sales volume responds to price changes. In stores with elastic demand, stock levels can be adjusted to meet spikes during price promotions, avoiding stockouts or excess inventory.

- Supplier Negotiations and Cost Control: If demand is elastic, Walmart must maintain competitive prices to avoid losing customers, so negotiating better wholesale costs becomes critical. For inelastic stores, Walmart has more flexibility in pricing and margin management.

In [26]:
df["snap_active"] = (
    (df["snap_CA"] + df["snap_TX"] + df["snap_WI"]) > 0
).astype(int)

naive_snap_effect = df.groupby("snap_active")["demand"].mean()
print("Naive SNAP demand:\n", naive_snap_effect)

Naive SNAP demand:
 snap_active
0    1.138703
1    1.245671
Name: demand, dtype: float64


In [27]:
model_df = df[["demand", "sell_price", "snap_active"]].copy()
model_df = model_df[(model_df["demand"] > 0) & (model_df["sell_price"] > 0)]

model_df["log_price"] = np.log(model_df["sell_price"])
X = model_df[["log_price", "snap_active"]]
X = sm.add_constant(X)
y = np.log(model_df["demand"])

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 demand   R-squared:                       0.100
Model:                            OLS   Adj. R-squared:                  0.100
Method:                 Least Squares   F-statistic:                 4.592e+05
Date:                Wed, 02 Jul 2025   Prob (F-statistic):               0.00
Time:                        20:03:25   Log-Likelihood:            -9.5697e+06
No. Observations:             8223297   AIC:                         1.914e+07
Df Residuals:                 8223294   BIC:                         1.914e+07
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           1.0808      0.001   1979.110      

**Discussion of SNAP Impact on Demand**

We analyzed the effect of SNAP (Supplemental Nutrition Assistance Program) activity on product demand using an Ordinary Least Squares (OLS) regression. The key results from the model are:

Coefficient for snap_active: +0.0379 (significant at p < 0.001)

Coefficient for log_price: -0.3421 (also significant)

Intercept (constant): 1.0808

R-squared: 0.10, indicating that 10% of demand variation is explained by price and SNAP activity combined.

What Does This Mean? The positive coefficient for snap_active (0.0379) means that when SNAP benefits are active, demand increases by roughly 3.8% on average (since the model is in log-linear form). This suggests that SNAP participation modestly boosts demand, likely by increasing consumers’ purchasing power.

The negative price coefficient (-0.3421) aligns with expectations: higher prices reduce demand, consistent with typical price elasticity findings.

The R-squared of 0.10 shows that while price and SNAP status explain some variation in demand, other factors (seasonality, promotions, store location, etc.) also play a big role.

**Business Implications for Walmart**

- Planning and Inventory Management

SNAP-active periods or areas with high SNAP participation should anticipate slightly increased demand. Walmart can adjust inventory to ensure availability and avoid stockouts, especially on staple items frequently purchased by SNAP recipients.

- Targeted Marketing and Promotions

Knowing that SNAP participation drives demand, Walmart could design marketing strategies or promotions aligned with SNAP benefit schedules to maximize sales during those times.

- Pricing Strategy

Since SNAP boosts demand, Walmart could evaluate whether modest price adjustments during SNAP-active periods might increase overall revenue without harming affordability for SNAP recipients.

- Store Location and Assortment Decisions

Walmart can use SNAP participation data to optimize product assortments in stores located in areas with high SNAP activity, ensuring products that benefit from SNAP-driven demand are well stocked.